# Gradio Chatbot

In [1]:
import gradio as gr

In [2]:
def count_images(message, history):
    num_images = len(message["files"])
    total_images = 0
    for message in history:
        if isinstance(message["content"], tuple):
            total_images += 1
    return f"You just uploaded {num_images} images, total uploaded: {total_images+num_images}"

In [22]:
demo = gr.ChatInterface(
    title="BioMED",
    description="## A description for the interface; if provided, appears above the chatbot and beneath the title in regular font. Accepts Markdown and HTML content.",
    theme="gradio/monochrome",
    show_progress="full",
    fill_height=True,
    fill_width=True,
    # save_history=True,
    fn=count_images, 
    type="messages", 
    # examples=[
    #     {"text": "No files", "files": []}
    # ], 
    multimodal=True,
    textbox=gr.MultimodalTextbox(file_count="multiple", file_types=["image", ".pdf"], sources=["upload"], interactive=True),
)

demo.launch()

* Running on local URL:  http://127.0.0.1:7877

To create a public link, set `share=True` in `launch()`.


In [23]:
# import gradio as gr

# def echo(message, history):
#     print("HISTORY", history, " | MESSAGE", message)
#     return message

# demo = gr.ChatInterface(fn=echo, type="messages", examples=["hello", "hola", "merhaba"], title="Echo Bot")
# demo.launch()

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


HISTORY []  | MESSAGE Hi
HISTORY [{'role': 'user', 'metadata': None, 'content': 'Hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hi', 'options': None}]  | MESSAGE How are you
HISTORY [{'role': 'user', 'metadata': None, 'content': 'Hi', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hi', 'options': None}, {'role': 'user', 'metadata': None, 'content': 'How are you', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'How are you', 'options': None}]  | MESSAGE Bye
